In [1]:
from collections import defaultdict

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from ydata_profiling import ProfileReport

from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from torchmetrics import MeanSquaredError, MeanAbsoluteError

import optuna

from pytorch_tools import CreateDataset, train_model_cls, train_model_reg, plot_metrics

In [2]:
df = pd.read_csv('data/ConcreteStrengthData.csv')

In [3]:
df

,CementComponent,BlastFurnaceSlag,FlyAshComponent,WaterComponent,SuperplasticizerComponent,CoarseAggregateComponent,FineAggregateComponent,AgeInDays,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


In [4]:
X = df.drop(columns=['Strength'])
y = df['Strength']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((824, 8), (206, 8), (824,), (206,))

In [5]:
train_dataset = CreateDataset(X_train, y_train)
test_dataset = CreateDataset(X_test, y_test)

train_dataloader = DataLoader(train_dataset,
                              batch_size=40,
                              num_workers=0
                             )

test_dataloader = DataLoader(test_dataset,
                              batch_size=40,
                              num_workers=0
                             )

In [ ]:
def objective(trial):
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    hidden1 = trial.suggest_int("hidden1", 64, 512)
    hidden2 = trial.suggest_int("hidden2", 32, hidden1)
    hidden3 = trial.suggest_int("hidden3", 16, hidden2)

    class LinearModel(nn.Module):
        def __init__(self, in_dim, out_dim=1):
            super().__init__()
            self.features = nn.Sequential(
                nn.Linear(in_dim, hidden1),
                nn.ReLU(),
                nn.Linear(hidden1, hidden2),
                nn.ReLU(),
                nn.Linear(hidden2, hidden3),
                nn.ReLU(),
                nn.Linear(hidden3, out_dim),
            )
        def forward(self, x):
            return self.features(x)

    model = LinearModel(in_dim=X_train.shape[1], out_dim=1)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    metrics, _ = train_model_reg(
        num_epoch=20,
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader,
        model=model,
        criterion=criterion,
        optimizer=optimizer,
        device=torch.device("cpu")
    )

    return metrics['test_mse'][-1]

In [7]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)

[I 2025-04-27 13:17:11,598] A new study created in memory with name: no-name-a83242ba-6f3b-4de1-b70b-f1c3ff931563


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch [1/20] Train Loss: 674.4020 MSE: 674.4019 MAE: 20.5117 RMSE: 25.9692
Epoch [2/20] Train Loss: 217.2241 MSE: 217.2242 MAE: 11.7490 RMSE: 14.7385
Epoch [3/20] Train Loss: 172.2679 MSE: 172.2680 MAE: 10.6052 RMSE: 13.1251
Epoch [4/20] Train Loss: 133.2340 MSE: 133.2340 MAE: 9.1982 RMSE: 11.5427
Epoch [5/20] Train Loss: 95.5971 MSE: 95.5971 MAE: 7.6080 RMSE: 9.7774
Epoch [6/20] Train Loss: 67.6205 MSE: 67.6206 MAE: 6.2668 RMSE: 8.2232
Epoch [7/20] Train Loss: 57.0490 MSE: 57.0490 MAE: 5.7529 RMSE: 7.5531
Epoch [8/20] Train Loss: 52.3336 MSE: 52.3336 MAE: 5.5285 RMSE: 7.2342
Epoch [9/20] Train Loss: 48.8065 MSE: 48.8065 MAE: 5.3078 RMSE: 6.9862
Epoch [10/20] Train Loss: 45.7356 MSE: 45.7356 MAE: 5.1230 RMSE: 6.7628
Epoch [11/20] Train Loss: 42.8161 MSE: 42.8161 MAE: 4.9457 RMSE: 6.5434
Epoch [12/20] Train Loss: 39.8449 MSE: 39.8449 MAE: 4.7656 RMSE: 6.3123
Epoch [13/20] Train Loss: 37.3371 MSE: 37.3371 MAE: 4.6179 RMSE: 6.1104
Epoch [14/20] Train Loss: 35.0141 MSE: 35.0141 MAE: 4.4775

[I 2025-04-27 13:17:12,521] Trial 0 finished with value: 51.51467514038086 and parameters: {'lr': 0.002488562295549242, 'hidden1': 388, 'hidden2': 334, 'hidden3': 163}. Best is trial 0 with value: 51.51467514038086.


Epoch [16/20] Train Loss: 31.3248 MSE: 31.3248 MAE: 4.2445 RMSE: 5.5969
Epoch [17/20] Train Loss: 29.6640 MSE: 29.6640 MAE: 4.1329 RMSE: 5.4465
Epoch [18/20] Train Loss: 28.2420 MSE: 28.2420 MAE: 4.0384 RMSE: 5.3143
Epoch [19/20] Train Loss: 26.9594 MSE: 26.9594 MAE: 3.9505 RMSE: 5.1922
Epoch [20/20] Train Loss: 25.8263 MSE: 25.8263 MAE: 3.8694 RMSE: 5.0820


...

Epoch [18/20] Train Loss: 41.1519 MSE: 41.1519 MAE: 4.8667 RMSE: 6.4150
Epoch [19/20] Train Loss: 39.4011 MSE: 39.4011 MAE: 4.7666 RMSE: 6.2770
Epoch [20/20] Train Loss: 37.7347 MSE: 37.7347 MAE: 4.6718 RMSE: 6.1429


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch [1/20] Train Loss: 1579.5020 MSE: 1579.5023 MAE: 35.9888 RMSE: 39.7430
Epoch [2/20] Train Loss: 1577.9804 MSE: 1577.9806 MAE: 35.9682 RMSE: 39.7238
Epoch [3/20] Train Loss: 1576.1620 MSE: 1576.1620 MAE: 35.9440 RMSE: 39.7009
Epoch [4/20] Train Loss: 1573.8133 MSE: 1573.8136 MAE: 35.9133 RMSE: 39.6713
Epoch [5/20] Train Loss: 1570.9803 MSE: 1570.9802 MAE: 35.8760 RMSE: 39.6356
Epoch [6/20] Train Loss: 1567.4948 MSE: 1567.4949 MAE: 35.8298 RMSE: 39.5916
Epoch [7/20] Train Loss: 1563.2547 MSE: 1563.2544 MAE: 35.7726 RMSE: 39.5380
Epoch [8/20] Train Loss: 1558.1701 MSE: 1558.1704 MAE: 35.7032 RMSE: 39.4737
Epoch [9/20] Train Loss: 1552.1199 MSE: 1552.1201 MAE: 35.6202 RMSE: 39.3970
Epoch [10/20] Train Loss: 1545.1413 MSE: 1545.1415 MAE: 35.5245 RMSE: 39.3083
Epoch [11/20] Train Loss: 1537.1404 MSE: 1537.1405 MAE: 35.4148 RMSE: 39.2064
Epoch [12/20] Train Loss: 1527.9950 MSE: 1527.9949 MAE: 35.2889 RMSE: 39.0896
Epoch [13/20] Train Loss: 1517.3751 MSE: 1517.3751 MAE: 35.1420 RMSE: 38.

[I 2025-04-27 13:17:18,539] Trial 9 finished with value: 1346.300048828125 and parameters: {'lr': 4.1063788757351374e-05, 'hidden1': 198, 'hidden2': 62, 'hidden3': 23}. Best is trial 1 with value: 47.28493881225586.


Epoch [19/20] Train Loss: 1421.9395 MSE: 1421.9395 MAE: 33.8052 RMSE: 37.7086
Epoch [20/20] Train Loss: 1399.9611 MSE: 1399.9612 MAE: 33.4917 RMSE: 37.4161


In [8]:
print("Best trial:")
print(study.best_trial.params)

Best trial:
{'lr': 0.0026931028539485162, 'hidden1': 78, 'hidden2': 65, 'hidden3': 57}


In [9]:
class Model(torch.nn.Module):
    def __init__(self, in_dim, out_dim=1):
        super().__init__()
        
        self.features = torch.nn.Sequential(
            nn.Linear(in_dim, 144),
            torch.nn.ReLU(),
            
            nn.Linear(144, 72),
            torch.nn.ReLU(),
            
            nn.Linear(72, 72),
            torch.nn.ReLU(),
            
            nn.Linear(72, out_dim),
        )
    
        
    def forward(self, x):
        output = self.features(x)
        return output
    

model = Model(in_dim=X_train.shape[1], out_dim=1)
  
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

metrics, model = train_model_reg(num_epoch=20,
                    train_dataloader = train_dataloader,
                    test_dataloader = test_dataloader,
                    model=model,
                    criterion=criterion,
                    optimizer=optimizer,
                    )

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch [1/20] Train Loss: 1410.4899 MSE: 1410.4900 MAE: 33.5491 RMSE: 37.5565
Epoch [2/20] Train Loss: 437.2239 MSE: 437.2240 MAE: 16.2158 RMSE: 20.9099
Epoch [3/20] Train Loss: 229.5909 MSE: 229.5909 MAE: 12.1970 RMSE: 15.1523
Epoch [4/20] Train Loss: 198.2681 MSE: 198.2681 MAE: 11.2724 RMSE: 14.0808
Epoch [5/20] Train Loss: 171.0141 MSE: 171.0141 MAE: 10.5859 RMSE: 13.0772
Epoch [6/20] Train Loss: 157.9729 MSE: 157.9729 MAE: 10.1697 RMSE: 12.5687
Epoch [7/20] Train Loss: 147.4507 MSE: 147.4507 MAE: 9.8212 RMSE: 12.1429
Epoch [8/20] Train Loss: 136.7180 MSE: 136.7180 MAE: 9.4535 RMSE: 11.6926
Epoch [9/20] Train Loss: 125.4474 MSE: 125.4474 MAE: 9.0245 RMSE: 11.2003
Epoch [10/20] Train Loss: 112.7358 MSE: 112.7357 MAE: 8.5197 RMSE: 10.6177
Epoch [11/20] Train Loss: 98.9476 MSE: 98.9476 MAE: 7.9256 RMSE: 9.9472
Epoch [12/20] Train Loss: 84.8928 MSE: 84.8928 MAE: 7.2799 RMSE: 9.2137
Epoch [13/20] Train Loss: 72.2589 MSE: 72.2589 MAE: 6.6278 RMSE: 8.5005
Epoch [14/20] Train Loss: 62.2958 M

In [10]:
y_true = []
y_pred = []

with torch.no_grad():
    for X_batch, y_batch in test_dataloader:
        predictions = model(X_batch).squeeze()
        
        y_true.extend(y_batch.numpy())
        y_pred.extend(predictions.numpy())

r2 = r2_score(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)

print(f"R²: {r2:.4f}")
print(f"MSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")

R²: 0.8139
MSE: 47.9617
MAE: 5.5596
